## 1. Load libraries, and set notebooks' path once for all

In [1]:
import itertools
import pandas as pd
import os, pyreadstat, re
from bs4 import BeautifulSoup

os.getcwd() # Make sure to be in the 'notebooks' folder

'C:\\Users\\pfaha\\PROJECTS\\brfss-diabetes-surveys\\brfss_diabetes_surveys_etl\\notebooks'

## 2. Load data, and its variables' description

In [2]:
df, _ = pyreadstat.read_xport('../data/raw/LLCP2024.xpt', encoding='windows-1252') 

In [3]:
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,CELPHON1,LADULT1,NUMADULT,RESPSLC1,LANDSEX3,SAFETIME,CTELNUM1,CELLFON5,CADULT1,CELLSEX3,PVTRESD3,CCLGHOUS,CSTATE1,LANDLINE,HHADULT,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINS2,PERSDOC3,MEDCOST1,CHECKUP1,EXERANY2,LASTDEN4,RMVTETH4,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNC1,CHCOCNC1,CHCCOPD3,ADDEPEV3,CHCKDNY2,HAVARTH4,DIABETE4,DIABAGE4,MARITAL,EDUCA,RENTHOM1,NUMHHOL4,NUMPHON4,CPDEMO1C,VETERAN3,EMPLOY1,CHILDREN,INCOME3,PREGNANT,WEIGHT2,HEIGHT3,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,HADMAM,HOWLONG,CERVSCRN,CRVCLCNC,CRVCLPAP,CRVCLHPV,HADHYST2,HADSIGM4,COLNSIGM,COLNTES1,SIGMTES1,LASTSIG4,...,_STRWT,_RAWRAKE,_WT2RAKE,_IMPRACE,_CHISPNC,_CRACE1,CAGEG,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT2,_LLCPWT,_RFHLTH,_PHYS14D,_MENT14D,_HLTHPL2,_HCVU654,_TOTINDA,_EXTETH3,_ALTETH3,_DENVST3,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR2,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACEPRV,_SEX,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG1,_RFMAM23,_MAM402Y,_CRVSCRN,_RFPAP37,_HPV5YR1,_PAPHPV1,_HADCOLN,_CLNSCP2,_HADSIGM,_SGMSCP2,_SGMS102,_RFBLDS6,_STOLDN2,_VIRCOL2,_SBONTI2,_CRCREC3,_SMOKER3,_RFSMOK3,_CURECI3,LCSLAST_,LCSNUMC_,_LCSAGE,_LCSYSMK,_PACKDAY,_PACKYRS,_LCSYQTS,_LCSSMKG,_LCSELIG,_LCSCTSN,_LCSPSTF,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK3,_RFDRHV9,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,2.0,02282024,02,28,2024,1100.0,2024000001,2.024000e+09,1.0,1.0,NaN,1.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,88.0,88.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,3.0,4.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,99.0,NaN,131.0,504.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,NaN,NaN,...,28.393219,1.0,28.393219,1.0,NaN,NaN,NaN,NaN,1.0,0.465294,220.149005,261.525511,1.0,2.0,1.0,1.0,9.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,12.0,2.0,78.0,6.0,64.0,163.0,5942.0,2249.0,2.0,1.0,1.0,2.0,9.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,6.0,2.0,NaN,9.0,2.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0
1,1.0,2.0,02212024,02,21,2024,1100.0,2024000002,2.024000e+09,1.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,88.0,88.0,NaN,3.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,1.0,6.0,1.0,2.0,NaN,1.0,1.0,7.0,88.0,11.0,NaN,180.0,510.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,4.0,NaN,NaN,...,28.393219,2.0,56.786438,1.0,NaN,NaN,NaN,NaN,1.0,0.465294,440.298009,307.169688,1.0,1.0,1.0,1.0,9.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,13.0,2.0,80.0,6.0,70.0,178.0,8165.0,2583.0,3.0,2.0,1.0,4.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,27.0,20.0,2.0,2.0,100.0,2.0,53.0,5.0,2.0,4.0,9.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0
2,1.0,2.0,02212024,02,21,2024,1100.0,2024000003,2.024000e+09,1.0,1.0,NaN,1.0,2.0,1.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,30.0,88.0,1.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,6.0,5.0,1.0,2.0,NaN,8.0,1.0,1.0,88.0,99.0,NaN,195.0,606.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,28.393219,3.0,85.179657,1.0,NaN,NaN,NaN,NaN,9.0,NaN,1938.892385,2939.862806,1.0,3.0,1.0,1.0,1.0,1.0,2.0,NaN,2.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,8.0,1.0,59.0,5.0,78.0,198.0,8845.0,2253.0,2.0,1.0,1.0,3.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,2.0,1.0,NaN,15.0,2.0,45.0,75.0,34.0,NaN,1.0,1.0,4.0,2.0,1.0,100.0,2.0,1400.0,1.0,NaN,NaN,2.0
3,1.0,2.0,02282024,02,28,2024,1100.0,2024000004,2.024000e+09,1.0,1.0,NaN,1.0,2.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,88.0,88.0,NaN,3.0,1.0,2.0,1.0,1.0,1.0,8.0,2.0,2.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,1.0,6.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,6.0,NaN,165.0,508.0,1.0,2.0,2.0,2.

In [4]:
df.shape

(457670, 301)

For detailed information on variable definitions and BRFSS 2024 data collection methodology, consult the [official 2024 BRFSS documentation](https://www.cdc.gov/brfss/annual_data/annual_2024.html).

The dataset contains post-processed responses from standardised questionnaires administered identically across all respondents. The BRFSS applied methodology ensures the final data is representative of each U.S. part considered.

**DIABETE4** is the main variable we want to study: it describes if a person is diagnosed as having diabetes, or prediabetes, or not at all.

In [5]:
# Parse HTML text of variables' description
def clean_html_text(text):
    # A quick fix on some exact non-breaking spaces & weird chars
    text = text.replace('\xa0', ' ').replace('�', '')
    # A normalisation of only the needed section headers
    text = re.sub(pattern=r'Section\s+Name:\s*', repl='Section Name: ', string=text, flags=re.IGNORECASE)
    text = re.sub(pattern=r'SAS\s+Variable\s+Name:\s*', repl='SAS Variable Name: ', string=text, flags=re.IGNORECASE)
    return text.strip()
def extract_html_text(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        text = soup.get_text(separator=' ')
        return clean_html_text(text)

text = extract_html_text('../docs/USCODE24_LLCP_082125.HTML')
text[:300]

'LLCP 2024: Codebook Report \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n             LLCP 2024 Codebook Report \n \n \n   \n \n \n Overall version data \n \n \n weighted with _LLCPWT \n \n \n Behavioral Risk Factor Surveillance System \n \n \n August 21, 2025 \n \n \n  \n  \n  \n  \n  \n  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n   \n \n \n   '

In [6]:
# Extract various section names
def extract_section_names(text):
    pattern = r'Section Name:\s*([^\n\r]+)'
    matches = re.findall(pattern, text, re.IGNORECASE)
    section_names = []
    for match in matches:
        truncated = match.split('Section Number')[0].split('Module Number')[0].split('Core')[0].strip()
        section_names.append(truncated)
    return sorted(list(set(section_names)))

names = extract_section_names(text)
assert len(names) > 0

# Extract all variables corresponding to a 'Section Name'
def extract_vars(text, pattern):
    escaped = re.escape(pattern)
    regex_pattern = rf'Section Name:\s*{escaped}.*?SAS Variable Name:\s*([A-Z0-9_]+)'
    matches = re.findall(regex_pattern, text, re.DOTALL | re.IGNORECASE)
    print(f"Pattern '{pattern}' → {len(matches)} vars")
    return sorted(list(set(matches)))

vars_per_section = {key: extract_vars(text, key) for key in names}
assert all(vars_per_section.values()), "Some sections have an empty list"

Pattern 'Adverse Childhood Experiences' → 13 vars
Pattern 'Alcohol Consumption' → 4 vars
Pattern 'Arthritis' → 1 vars
Pattern 'Breast and Cervical Cancer Screening' → 7 vars
Pattern 'Calculated Race Variables' → 6 vars
Pattern 'Calculated Variables' → 66 vars
Pattern 'Cancer Survivorship: Course of Treatment' → 8 vars
Pattern 'Cancer Survivorship: Pain Management' → 2 vars
Pattern 'Cancer Survivorship: Type of Cancer' → 3 vars
Pattern 'Caregiver' → 9 vars
Pattern 'Cell Phone Introduction' → 10 vars
Pattern 'Child Demographic Variables' → 2 vars
Pattern 'Child Weighting Variables' → 2 vars
Pattern 'Childhood Asthma Prevalence' → 2 vars
Pattern 'Chronic Health Conditions' → 13 vars
Pattern 'Cognitive Decline' → 5 vars
Pattern 'Colorectal Cancer Screening' → 13 vars
Pattern 'Demographics' → 13 vars
Pattern 'Diabetes' → 7 vars
Pattern 'Disability' → 6 vars
Pattern 'Exercise (Physical Activity)' → 1 vars
Pattern 'Family Planning' → 4 vars
Pattern 'Firearm Safety' → 3 vars
Pattern 'HIV/AIDS'

In [7]:
# Make sure that only variables from the text are being studied later
# Others which are not from the text do not have meaning we can be based on
expected_cols = set(sorted(list(itertools.chain.from_iterable(vars_per_section.values()))))
current_cols = set(sorted(df.columns))
if len(current_cols) > len(expected_cols):
    print(f"Expected cols total : {len(expected_cols)}")
    print(f"Current cols total : {len(current_cols)}")
    true_cols = list(current_cols & expected_cols)
    df = df[sorted(list(true_cols))]
    print(f"Keeping {len(true_cols)} columns in df...")

vars_to_keep_per_section = {
    key: [col for col in list_values if col in df.columns] 
    for key, list_values in vars_per_section.items() 
    if any(col in df.columns for col in list_values)
}

assert len(list(itertools.chain.from_iterable(vars_to_keep_per_section.values()))) == df.shape[1]
assert 'DIABETE4' in df.columns

Expected cols total : 297
Current cols total : 301
Keeping 296 columns in df...


## 3. Try some preprocessing

In [8]:
df1 = df.copy()

In [9]:
df1.head()

,ACEADNED,ACEADSAF,ACEDEPRS,ACEDIVRC,ACEDRINK,ACEDRUGS,ACEHURT1,ACEHVSEX,ACEPRISN,ACEPUNCH,ACESWEAR,ACETOUCH,ACETTHEM,ADDEPEV3,ALCDAY4,ARTHEXER,ASTHMA3,ASTHNOW,AVEDRNK4,BLDSTFIT,BLIND,CADULT1,CAGEG,CAREGIV1,CASTHDX2,CASTHNO2,CCLGHOUS,CDDISCU1,CDHOUS1,CDSOCIA1,CDWORRY,CELLFON5,CELLSEX3,CELPHON1,CERVSCRN,CHCCOPD3,CHCKDNY2,CHCOCNC1,CHCSCNC1,CHECKUP1,CHILDREN,CHKHEMO3,CIMEMLO1,CNCRAGE,CNCRDIFF,CNCRTYP2,COLGHOUS,COLNCNCR,COLNSIGM,COLNTES1,CPDEMO1C,CRGVALZD,CRGVHOU2,CRGVHRS2,CRGVLNG2,CRGVNURS,CRGVPER2,CRGVPRB4,CRGVREL5,CRVCLCNC,CRVCLHPV,CRVCLPAP,CSRVCLIN,CSRVCTL2,CSRVDEIN,CSRVDOC1,CSRVINSR,CSRVINST,CSRVPAIN,CSRVRTRN,CSRVSUM,CSRVTRT3,CSTATE1,CTELENM1,CTELNUM1,CVDCRHD4,CVDINFR4,CVDSTRK3,DEAF,DECIDE,DIABAGE4,DIABEDU1,DIABETE4,DIABEYE1,...,VIRCOLO1,WEIGHT2,WTKG3,_AGE65YR,_AGE80,_AGEG5YR,_AGE_G,_AIDTST4,_ALTETH3,_ASTHMS1,_BMI5,_BMI5CAT,_CASTHM1,_CHISPNC,_CHLDCNT,_CLLCPWT,_CLNSCP2,_CRACE1,_CRCREC3,_CRVSCRN,_CURECI3,_DENVST3,_DRDXAR2,_DRNKWK3,_DUALCOR,_DUALUSE,_EDUCAG,_EXTETH3,_FLSHOT7,_HADCOLN,_HADSIGM,_HCVU654,_HISPANC,_HLTHPL2,_HPV5YR1,_IMPRACE,_INCOMG1,_LCSAGE,_LCSCTSN,_LCSELIG,_LCSPSTF,_LCSSMKG,_LCSYQTS,_LCSYSMK,_LLCPWT,_LLCPWT2,_LTASTH1,_MAM402Y,_MENT14D,_METSTAT,_MICHD,_MRACE1,_PACKDAY,_PACKYRS,_PAPHPV1,_PHYS14D,_PNEUMO3,_PSU,_RACE,_RACEG21,_RACEGR3,_RACEPRV,_RAWRAKE,_RFBING6,_RFBLDS6,_RFBMI5,_RFDRHV9,_RFHLTH,_RFMAM23,_RFPAP37,_RFSMOK3,_SBONTI2,_SEX,_SGMS102,_SGMSCP2,_SMOKER3,_STATE,_STOLDN2,_STRWT,_STSTR,_TOTINDA,_URBSTAT,_VIRCOL2,_WT2RAKE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,888.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,2.0,1.0,1.0,88.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,2.0,2.0,2.0,2.0,NaN,NaN,3.0,NaN,...,1.0,131.0,5942.0,2.0,78.0,12.0,6.0,2.0,1.0,3.0,2249.0,2.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.465294,1.0,2.0,2.0,1.0,1.0,2.0,9.0,2.0,1.0,NaN,1.0,9.0,2.0,NaN,2.0,9.0,6.0,NaN,NaN,261.525511,220.149005,1.0,NaN,1.0,1.0,2.0,1.0,NaN,NaN,NaN,2.0,2.0,2.024000e+09,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,2.0,NaN,NaN,4.0,1.0,NaN,28.393219,11011.0,1.0,1.0,NaN,28.393219
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,888.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,2.0,2.0,1.0,88.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,1.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,2.0,2.0,2.0,NaN,NaN,3.0,NaN,...,2.0,180.0,8165.0,2.0,80.0,13.0,6.0,2.0,1.0,3.0,2583.0,3.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.465294,1.0,4.0,2.0,1.0,1.0,2.0,9.0,2.0,1.0,NaN,1.0,7.0,2.0,4.0,2.0,9.0,5.0,53.0,2.0,307.169688,440.298009,1.0,NaN,1.0,1.0,1.0,1.0,100.0,2.0,NaN,1.0,1.0,2.024000e+09,1.0,1.0,1.0,1.0,2.0,1.0,NaN,2.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,3.0,1.0,NaN,28.393219,11011.0,1.0,1.0,NaN,56.786438
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,230.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,2.0,2.0,4.0,88.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,2.0,2.0,2.0,2.0,NaN,NaN,3.0,NaN,...,NaN,195.0,8845.0,1.0,59.0,8.0,5.0,2.0,NaN,3.0,2253.0,2.0,1.0,NaN,1.0,NaN,3.0,NaN,3.0,NaN,1.0,2.0,1.0,1400.0,NaN,9.0,3.0,2.0,NaN,2.0,2.0,1.0,2.0,1.0,NaN,1.0,9.0,2.0,4.0,1.0,2.0,1.0,NaN,45.0,2939.862806,1938.892385,1.0,NaN,1.0,2.0,2.0,1.0,75.0,34.0,NaN,3.0,NaN,2.024000e+09,1.0,1.0,1.0,1.0,3.0,2.0,3.0,1.0,1.0,1.0,NaN,NaN,2.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,28.393219,11011.0,1.0,1.0,3.0,85.179657
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,888.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,2.0,1.0,1.0,88.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0,3.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,2.0,2.0,1.0,2.0,NaN,Na

### 3.1. Remove missing values

In [10]:
(df1.isnull().sum()/len(df1)).describe() # Only less than 25% of the data has non-missing values

count    296.000000
mean       0.474914
std        0.406351
min        0.000000
25%        0.000013
50%        0.552399
75%        0.914434
max        0.999969
dtype: float64

In [11]:
# Any variable with an extremely high proportion of missing values is excluded from any further study
# First quartile is chosen as the threshold from which an extremely high proportion of missing values cannot be accepted
NA_THRESHOLD = 0.000015
cols_na_prop = {col: df1[col].isnull().sum()/len(df1) for col in df1.columns}
cols_to_remove = [col for col in cols_na_prop.keys() if cols_na_prop[col] >= NA_THRESHOLD]

print(f"Removing {len(cols_to_remove)} columns having at least {NA_THRESHOLD:.4%} of NA values...")
df1 = df1.drop(columns=cols_to_remove)

Removing 221 columns having at least 0.0015% of NA values...


In [12]:
vars_to_keep_per_section = {
    key: [col for col in list_values if col in df1.columns] 
    for key, list_values in vars_to_keep_per_section.items() 
    if any(col in df1.columns for col in list_values)
}
assert len(list(itertools.chain.from_iterable(vars_to_keep_per_section.values()))) == df1.shape[1]
assert 'DIABETE4' in df1.columns

In [13]:
vars_to_keep_per_section.keys()

dict_keys(['Calculated Race Variables', 'Calculated Variables', 'Chronic Health Conditions', 'Exercise (Physical Activity)', 'Health Care Access', 'Health Status', 'Healthy Days', 'Oral Health', 'Questionnaire Language', 'Questionnaire Version', 'Record Identification', 'Respondent Sex', 'Weighting Variables'])

### 3.3. Remove any near constant variable if there's

In [14]:
len(df1) # Total samples

457670

In [15]:
pd.Series({col: df1[col].value_counts(dropna=False).iloc[0] for col in df1.columns}).describe()

count        75.000000
mean     263653.386667
std      140053.114956
min          53.000000
25%      197405.000000
50%      296778.000000
75%      377339.500000
max      438117.000000
dtype: float64

In [16]:
pd.Series({col: df1[col].value_counts(normalize=True, dropna=False).iloc[0] for col in df1.columns}).describe()

count    75.000000
mean      0.576077
std       0.306013
min       0.000116
25%       0.431326
50%       0.648454
75%       0.824479
max       0.957277
dtype: float64

We have only variables with a dominant value proportion of 96% at most, which is equal to 438117 over a total of 457670.

In [17]:
vars_to_keep_per_section.keys()

dict_keys(['Calculated Race Variables', 'Calculated Variables', 'Chronic Health Conditions', 'Exercise (Physical Activity)', 'Health Care Access', 'Health Status', 'Healthy Days', 'Oral Health', 'Questionnaire Language', 'Questionnaire Version', 'Record Identification', 'Respondent Sex', 'Weighting Variables'])

### 3.4. Remove some unnecessary variables

**Calculated Race Variables: especially those, since we don't want any race-related misinterpretation later.**

In [18]:
vars_to_keep_per_section['Calculated Race Variables']

['_HISPANC', '_MRACE1', '_RACE', '_RACEG21', '_RACEGR3', '_RACEPRV']

In [19]:
cols_to_remove = vars_to_keep_per_section['Calculated Race Variables'].copy()

print(f"Removing {len(cols_to_remove)} race-related variables...")
df1 = df1.drop(columns=cols_to_remove)

Removing 6 race-related variables...


**Calculated Variables: None is race-related, and they could be useful.**

In [20]:
vars_to_keep_per_section['Calculated Variables']

['DRNKANY6',
 'DROCDY4_',
 '_AGE65YR',
 '_AGE80',
 '_AGEG5YR',
 '_AGE_G',
 '_ASTHMS1',
 '_CASTHM1',
 '_CHLDCNT',
 '_CURECI3',
 '_DENVST3',
 '_DRNKWK3',
 '_EDUCAG',
 '_EXTETH3',
 '_HCVU654',
 '_HLTHPL2',
 '_INCOMG1',
 '_LCSAGE',
 '_LTASTH1',
 '_MENT14D',
 '_PHYS14D',
 '_RFBING6',
 '_RFBMI5',
 '_RFDRHV9',
 '_RFHLTH',
 '_RFSMOK3',
 '_SEX',
 '_SMOKER3',
 '_TOTINDA']

**Chronic Health Conditions: None is race-related, but we're currently focusing only on 'DIABETE4'.**

In [21]:
vars_to_keep_per_section['Chronic Health Conditions']

['ADDEPEV3',
 'ASTHMA3',
 'CHCKDNY2',
 'CHCOCNC1',
 'CHCSCNC1',
 'CVDCRHD4',
 'CVDINFR4',
 'CVDSTRK3',
 'DIABETE4',
 'HAVARTH4']

In [22]:
cols_to_remove = [col for col in vars_to_keep_per_section['Chronic Health Conditions'] if col != 'DIABETE4']

print(f"Removing {len(cols_to_remove)} chronic health conditions variables...")
df1 = df1.drop(columns=cols_to_remove)

Removing 9 chronic health conditions variables...


**Exercise (Physical Activity): The related variable expresses physical health over the last 30 days. We won't keep it.**

In [23]:
vars_to_keep_per_section['Exercise (Physical Activity)']

['EXERANY2']

In [24]:
cols_to_remove = vars_to_keep_per_section['Exercise (Physical Activity)'].copy()

print(f"Removing {len(cols_to_remove)} physical activity variable...")
df1 = df1.drop(columns=cols_to_remove)

Removing 1 physical activity variable...


**Health Care Access: They all seem relevant to associate with 'DIABETE4'. We keep them all.**

In [25]:
vars_to_keep_per_section['Health Care Access']

['CHECKUP1', 'MEDCOST1', 'PERSDOC3', 'PRIMINS2']

**Health Status: The related variable gives an overview of the interviewed person's general health status. It's his own vision of himself. We keep it.**

In [26]:
vars_to_keep_per_section['Health Status']

['GENHLTH']

**Healthy Days: The related variables give an overview of the number of days the interviewed person's physical or mental health was not good. We keep them.**

In [27]:
vars_to_keep_per_section['Healthy Days']

['MENTHLTH', 'PHYSHLTH']

**Oral Health: The related variables for a person's dental health. We keep them.**

In [28]:
vars_to_keep_per_section['Oral Health']

['LASTDEN4', 'RMVTETH4']

**Questionnaire Language: The related variable seems irrelevant for our study.**

In [29]:
vars_to_keep_per_section['Questionnaire Language']

['QSTLANG']

In [30]:
cols_to_remove = vars_to_keep_per_section['Questionnaire Language'].copy()

print(f"Removing {len(cols_to_remove)} questionnaire language variable...")
df1 = df1.drop(columns=cols_to_remove)

Removing 1 questionnaire language variable...


**Questionnaire Version: The related variable seems irrelevant for our study.**

In [31]:
vars_to_keep_per_section['Questionnaire Version']

['QSTVER']

In [32]:
cols_to_remove = vars_to_keep_per_section['Questionnaire Version'].copy()

print(f"Removing {len(cols_to_remove)} questionnaire version variable...")
df1 = df1.drop(columns=cols_to_remove)

Removing 1 questionnaire version variable...


**Record Identification: All related variables seem irrelevant for our study, except '_STATE'.**

In [33]:
vars_to_keep_per_section['Record Identification']

['DISPCODE',
 'FMONTH',
 'IDATE',
 'IDAY',
 'IMONTH',
 'IYEAR',
 'SEQNO',
 '_PSU',
 '_STATE']

In [34]:
cols_to_remove = [col for col in vars_to_keep_per_section['Record Identification'] if col not in ['_STATE', 'DISPCODE']]

print(f"Removing {len(cols_to_remove)} record identification variables...")
df1 = df1.drop(columns=cols_to_remove)

Removing 7 record identification variables...


**Respondent Sex: The variable contains the same information as the variable '_SEX', which is already kept, so we remove this one.**

In [35]:
vars_to_keep_per_section['Respondent Sex']

['SEXVAR']

In [36]:
cols_to_remove = vars_to_keep_per_section['Respondent Sex'].copy()

print(f"Removing {len(cols_to_remove)} respondent sex variable...")
df1 = df1.drop(columns=cols_to_remove)

Removing 1 respondent sex variable...


**Weighting Variables: Most have hidden values or race-related information, or information we do not currently know how to use. We exclude all of them.**

In [37]:
vars_to_keep_per_section['Weighting Variables']

['_DUALUSE',
 '_IMPRACE',
 '_LLCPWT',
 '_LLCPWT2',
 '_RAWRAKE',
 '_STRWT',
 '_STSTR',
 '_WT2RAKE']

In [38]:
cols_to_remove = vars_to_keep_per_section['Weighting Variables'].copy()

print(f"Removing {len(cols_to_remove)} weighting variables...")
df1 = df1.drop(columns=cols_to_remove)

Removing 8 weighting variables...


In [39]:
vars_to_keep_per_section = {
    key: [col for col in list_values if col in df1.columns] 
    for key, list_values in vars_to_keep_per_section.items() 
    if any(col in df1.columns for col in list_values)
}
assert len(list(itertools.chain.from_iterable(vars_to_keep_per_section.values()))) == df1.shape[1]
assert 'DIABETE4' in df1.columns

In [40]:
vars_to_keep_per_section

{'Calculated Variables': ['DRNKANY6',
  'DROCDY4_',
  '_AGE65YR',
  '_AGE80',
  '_AGEG5YR',
  '_AGE_G',
  '_ASTHMS1',
  '_CASTHM1',
  '_CHLDCNT',
  '_CURECI3',
  '_DENVST3',
  '_DRNKWK3',
  '_EDUCAG',
  '_EXTETH3',
  '_HCVU654',
  '_HLTHPL2',
  '_INCOMG1',
  '_LCSAGE',
  '_LTASTH1',
  '_MENT14D',
  '_PHYS14D',
  '_RFBING6',
  '_RFBMI5',
  '_RFDRHV9',
  '_RFHLTH',
  '_RFSMOK3',
  '_SEX',
  '_SMOKER3',
  '_TOTINDA'],
 'Chronic Health Conditions': ['DIABETE4'],
 'Health Care Access': ['CHECKUP1', 'MEDCOST1', 'PERSDOC3', 'PRIMINS2'],
 'Health Status': ['GENHLTH'],
 'Healthy Days': ['MENTHLTH', 'PHYSHLTH'],
 'Oral Health': ['LASTDEN4', 'RMVTETH4'],
 'Record Identification': ['DISPCODE', '_STATE']}

### 3.5. Duplicated Rows

In [41]:
# Remove any eventual redundant row
print(f"Rows before deduplication: {len(df1)}")
df1 = df1.drop_duplicates()
print(f"Rows after deduplication: {len(df1)}")

Rows before deduplication: 457670
Rows after deduplication: 457425


In [42]:
df1.shape

(457425, 41)

In [43]:
df1.head()

,CHECKUP1,DIABETE4,DISPCODE,DRNKANY6,DROCDY4_,GENHLTH,LASTDEN4,MEDCOST1,MENTHLTH,PERSDOC3,PHYSHLTH,PRIMINS2,RMVTETH4,_AGE65YR,_AGE80,_AGEG5YR,_AGE_G,_ASTHMS1,_CASTHM1,_CHLDCNT,_CURECI3,_DENVST3,_DRNKWK3,_EDUCAG,_EXTETH3,_HCVU654,_HLTHPL2,_INCOMG1,_LCSAGE,_LTASTH1,_MENT14D,_PHYS14D,_RFBING6,_RFBMI5,_RFDRHV9,_RFHLTH,_RFSMOK3,_SEX,_SMOKER3,_STATE,_TOTINDA
0,1.0,3.0,1100.0,2.0,0.0,3.0,1.0,2.0,88.0,2.0,2.0,3.0,1.0,2.0,78.0,12.0,6.0,3.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,9.0,1.0,9.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0
1,1.0,3.0,1100.0,2.0,0.0,1.0,1.0,2.0,88.0,1.0,88.0,3.0,1.0,2.0,80.0,13.0,6.0,3.0,1.0,1.0,1.0,1.0,0.0,4.0,2.0,9.0,1.0,7.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
2,4.0,3.0,1100.0,1.0,100.0,2.0,4.0,1.0,88.0,3.0,30.0,1.0,2.0,1.0,59.0,8.0,5.0,3.0,1.0,1.0,1.0,2.0,1400.0,3.0,2.0,1.0,1.0,9.0,2.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
3,1.0,3.0,1100.0,2.0,0.0,1.0,1.0,2.0,88.0,1.0,88.0,3.0,8.0,2.0,80.0,13.0,6.0,3.0,1.0,1.0,1.0,1.0,0.0,4.0,1.0,9.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0
4,1.0,3.0,1100.0,2.0,0.0,3.0,1.0,2.0,88.0,1.0,88.0,5.0,1.0,1.0,47.0,6.0,4.0,3.0,1.0,1.0,1.0,1.0,0.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,2.0


**We will consider 40 characteristics for 457,425 individuals to explain their state of health regarding diabetes and prediabetes.**

**ETL Code Structure**
- Basic Units :
  - DataLoader (Load raw data)
  - DataCleaner (Perform transformations on raw data loaded)
  - DataSinker (Save the transformed raw data in a local folder)
- Full Orchestration Unit :
  - ETLPipeline (From data loading to cleaning, then saving results locally)